In [0]:
from pyspark.sql.functions import to_timestamp,hour,minute,when,col,current_timestamp,date_format
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [0]:
p1_weather_df = spark.table('solarPowerdata.plant1_weather_sensor_data')

In [0]:
display(p1_weather_df.summary())

summary,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
count,3182,3182,3182,3182,3182,3182
mean,null,4135001.0,null,25.5316057175644,31.091015378426455,0.228312830065651
stddev,null,0.0,null,3.354855664435953,12.261221623078292,0.3008363668152484
min,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,20.398504866666666,18.140415466666663,0.0
25%,null,4135001,null,22.704483933333336,21.08964233333333,0.0
50%,null,4135001,null,24.6126954,24.617095866666666,0.024557107333333335
75%,null,4135001,null,27.920831266666664,41.316923333333335,0.4497933986666667
max,2020-06-17 23:45:00,4135001,HmiyD2TTLFNqkNe,35.25248613333334,65.54571366666664,1.2216518466666668


In [0]:
p1_gen_df = spark.table('SolarPowerData.plant1_generation_data')

In [0]:
display(p1_gen_df.summary())

summary,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
count,68778,68778,68778,68778,68778,68778,68778
mean,null,4135001.0,null,3147.4262112269307,307.802752265517,3295.9687371825785,6978711.7606710475
stddev,null,0.0,null,4036.457168799247,394.3964386953456,3145.1783088786433,416271.98285620863
min,01-06-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6183645.0
25%,null,4135001,null,0.0,0.0,0.0,6512002.0
50%,null,4135001,null,427.625,41.3375,2658.0,7146685.0
75%,null,4135001,null,6365.375,623.4625,6274.0,7268656.571
max,31-05-2020 23:45,4135001,zVJPv84UY57bAof,14471.125,1410.95,9163.0,7846821.0


In [0]:
display(p1_gen_df.groupBy('SOURCE_KEY').count())

SOURCE_KEY,count
1BY6WEcLGh8j5v7,3154
adLQvlD726eNBSB,3119
rGa61gmuvPhdLxV,3124
YxYtjZvoooNbGkE,3104
pkci93gMrogZuBj,3125
bvBOhCH3iADSZry,3155
sjndEbLyjtCKgGv,3124
zVJPv84UY57bAof,3124
VHMLBKoKgIrUVDU,3133
uHbuxQJl8lW7ozc,3125


In [0]:
display(p1_gen_df)

DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0
15-05-2020 00:00,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0
15-05-2020 00:00,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7028673.0
15-05-2020 00:00,4135001,ZnxXDlPa8U1GXgE,0.0,0.0,0.0,6522172.0
15-05-2020 00:00,4135001,ZoEaEvLYb1n2sOq,0.0,0.0,0.0,7098099.0
15-05-2020 00:00,4135001,adLQvlD726eNBSB,0.0,0.0,0.0,6271355.0


In [0]:
p1_gen_df = p1_gen_df.withColumn('DATE_TIME',to_timestamp(p1_gen_df.DATE_TIME,'dd-MM-yyyy HH:mm'))
p1_weather_df = p1_weather_df.withColumn('DATE_TIME',to_timestamp(p1_weather_df.DATE_TIME,'yyyy-MM-dd HH:mm:ss'))

In [0]:
display(p1_weather_df)

DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
2020-05-15T00:00:00.000+0000,4135001,HmiyD2TTLFNqkNe,25.184316133333333,22.8575074,0.0
2020-05-15T00:15:00.000+0000,4135001,HmiyD2TTLFNqkNe,25.08458866666667,22.761667866666663,0.0
2020-05-15T00:30:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.935752600000004,22.592305533333334,0.0
2020-05-15T00:45:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.8461304,22.360852133333335,0.0
2020-05-15T01:00:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.621525357142858,22.165422642857145,0.0
2020-05-15T01:15:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.536092200000002,21.968570866666667,0.0
2020-05-15T01:30:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.638673866666664,22.352925666666668,0.0
2020-05-15T01:45:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.873022333333335,23.1609192,0.0
2020-05-15T02:00:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.936930466666663,23.026113,0.0
2020-05-15T02:15:00.000+0000,4135001,HmiyD2TTLFNqkNe,25.012247600000002,23.343229266666665,0.0


In [0]:
p1_df = p1_weather_df.select('DATE_TIME','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION').join(p1_gen_df, on = 'DATE_TIME',how = 'inner')

In [0]:
display(p1_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7028673.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZnxXDlPa8U1GXgE,0.0,0.0,0.0,6522172.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZoEaEvLYb1n2sOq,0.0,0.0,0.0,7098099.0
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,adLQvlD726eNBSB,0.0,0.0,0.0,6271355.0


In [0]:
p1_df = p1_df.withColumn('TIME',date_format('DATE_TIME','HH:mm:ss'))

In [0]:
# start ='06:30:00'
# end = '17:30:00'
# p1_df = p1_df.withColumn('Status',when((col('TIME')>=start) & (col('DATE_TIME')<=end) & (col('DC_POWER') == 0),'Fault').when((col('DATE_TIME')>start) & (col('DATE_TIME')<=end) & (col('DC_POWER') != 0),'Normal').otherwise('0'))

In [0]:
display(p1_df.filter(col('DC_POWER')==0))

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,TIME
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7028673.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZnxXDlPa8U1GXgE,0.0,0.0,0.0,6522172.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZoEaEvLYb1n2sOq,0.0,0.0,0.0,7098099.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,adLQvlD726eNBSB,0.0,0.0,0.0,6271355.0,00:00:00


In [0]:
display(p1_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,TIME
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7028673.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZnxXDlPa8U1GXgE,0.0,0.0,0.0,6522172.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZoEaEvLYb1n2sOq,0.0,0.0,0.0,7098099.0,00:00:00
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,adLQvlD726eNBSB,0.0,0.0,0.0,6271355.0,00:00:00


In [0]:
va = VectorAssembler().setInputCols(['IRRADIATION']).setOutputCol('features')
p1_df = va.transform(p1_df)

In [0]:
p1_df = p1_df.withColumnRenamed('AC_POWER','label')
display(p1_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,label,DAILY_YIELD,TOTAL_YIELD,TIME,features
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7028673.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZnxXDlPa8U1GXgE,0.0,0.0,0.0,6522172.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,ZoEaEvLYb1n2sOq,0.0,0.0,0.0,7098099.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"
2020-05-15T00:00:00.000+0000,25.184316133333333,22.8575074,0.0,4135001,adLQvlD726eNBSB,0.0,0.0,0.0,6271355.0,00:00:00,"Map(vectorType -> dense, length -> 1, values -> List(0.0))"


In [0]:
lr  = LinearRegression().setFeaturesCol('features').setLabelCol('label')

In [0]:
lr_model = lr.fit(p1_df)

In [0]:
lr_model.summary.meanAbsoluteError

Out[125]: 26.759621776249922

In [0]:
lr_model.setPredictionCol('ACPower_Predicted')

Out[126]: LinearRegressionModel: uid=LinearRegression_cc5b1b0d921d, numFeatures=1

In [0]:
pred_p1_df = lr_model.transform(p1_df)

In [0]:
pred_p1_df = pred_p1_df.withColumn('residuals',col('ACPower_Predicted') - col('label') )

In [0]:
outlier_residual_limit = pred_p1_df.stat.approxQuantile('residuals',[0.90],0.05)

In [0]:
p2_weather_df = spark.table('solarPowerdata.plant2_weather_sensor_data')
p2_gen_df = spark.table('solarPowerdata.plant2_generation_data')

In [0]:
p2_df = p2_weather_df.select('DATE_TIME','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION').join(p2_gen_df, on = 'DATE_TIME',how = 'inner')

In [0]:
display(p2_df.filter(col('DC_POWER')!=0))

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,4UPUqMRk7TRMgml,15.406666666666665,14.86,0.7999999999999999,2429011.8000000003
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,81aHJ1q11NBPMrL,14.773333333333333,14.246666666666668,0.7333333333333335,1.2152787367333333E9
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,9kRcWv60rDACzjR,15.38,14.84,1.8666666666666667,2.247719578866667E9
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,Et9kgGMDl729KT4,14.973333333333334,14.440000000000001,1.0,1704251.0000000002
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,IQ2d7wF4YD8zU1Q,15.16,14.633333333333338,0.7333333333333333,1.9941526733333334E7
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,LYwnQax7tkwH5Cb,15.566666666666665,15.02,1.2666666666666666,1.7949586352666667E9
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,LlT2YUhhzqhg5Sw,15.179999999999998,14.673333333333336,1.0666666666666667,2.8259281106666666E8
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,Mx2yZCDsyf6DPfv,15.1,14.586666666666668,0.7999999999999999,2453646.8
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,NgDl19wMapZy17u,14.98666666666667,14.453333333333335,1.2666666666666666,1.1151259226666668E8
2020-05-15 06:00:00,24.74278656666667,24.077229833333334,0.012961978146666667,4136001,PeE6FRyGXUgsRhN,14.506666666666668,13.993333333333334,0.6666666666666666,1.348350801666667E9


In [0]:

p2_df = va.transform(p2_df)
p2_df = p2_df.withColumnRenamed('AC_POWER','label')
pred_p2_df = lr_model.transform(p2_df)


In [0]:
pred_p2_df = pred_p2_df.withColumn('residuals',col('ACPower_Predicted')-col('label'))
fault_instances_df=  pred_p2_df.filter(col('residuals')>=pred_p2_df.stat.approxQuantile('residuals',[0.95],0.05)[0])

In [0]:
pred_p2_df.count()

Out[145]: 67698

In [0]:
fault_instances_df.count()

Out[146]: 7567